### Vector stores and retrievers
 familiarize with LangChain's vector store and retriever abstractions. These abstractions are designed to support retrieval of data-- from (vector) databases and other sources-- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.

### Documents
LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content;
- metadata: a dict containing arbitrary metadata.
The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.


In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

In [4]:
model = ChatGroq(model = 'llama-3.3-70b-versatile')
embedding = OllamaEmbeddings(model='embeddinggemma:300m')
vectordb = Chroma.from_documents(documents=documents,embedding=embedding)

In [5]:
vectordb

In [6]:
vectordb.similarity_search('Are Cats independent ?')

[Document(id='83225114-e9b3-4ac0-93c2-0a9af5ba1970', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='338cf05f-e049-402f-88c5-c969951fbb32', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(id='a6881920-326e-48bd-906c-d056b6269253', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='e84d12c8-50b9-4875-b33a-38ae2700f59e', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [7]:
vectordb.similarity_search_with_score('Are Cats independent ?')

[(Document(id='83225114-e9b3-4ac0-93c2-0a9af5ba1970', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.2336980253458023),
 (Document(id='338cf05f-e049-402f-88c5-c969951fbb32', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  0.37736010551452637),
 (Document(id='a6881920-326e-48bd-906c-d056b6269253', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.43681079149246216),
 (Document(id='e84d12c8-50b9-4875-b33a-38ae2700f59e', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  0.5216503143310547)]

### Retrievers
LangChain VectorStore objects are not Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

Where as LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

In [10]:
from langchain_core.runnables import RunnableLambda
#one way to create retrival (not generally recommended)
retriver = RunnableLambda(vectordb.similarity_search).bind(k=1)
retriver.batch(['Are cats independent?'])

[[Document(id='83225114-e9b3-4ac0-93c2-0a9af5ba1970', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]]

In [11]:
retrival = vectordb.as_retriever(
    search_type = 'similarity',
    search_kwargs = {'k':1}
)
retrival.batch(['Are cats independent?'])

[[Document(id='83225114-e9b3-4ac0-93c2-0a9af5ba1970', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]]

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer the below with the provided context only :
{question}

context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([('human',message)])
rag_chain = {'question':RunnablePassthrough(),'context':retrival}|prompt|model

response = rag_chain.invoke('which animal is the most social?')
response.content

'Based on the provided context, the answer is: Rabbits. They are described as "social animals".'